In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from textblob import TextBlob
import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.shape

(404290, 6)

In [4]:
new_df = df.sample(50000, random_state=12)

In [5]:
new_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
162260,162260,252637,78559,How can I accomplish more every single day?,How do I get motivated every single day?,0
300139,300139,422897,422898,How does GravityLight work?,How does GravityLight (From Deciwatt) work? Wh...,1
169568,169568,187323,74117,Why should I not vote for Trump? (Someone aske...,Why should I not vote for Trump?,1


In [6]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
new_df.duplicated().sum()

0

In [8]:
# for lemmatization
word_lemmatize = WordNetLemmatizer()

In [11]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()
    
    # spelling correction
    q = " ".join([word_lemmatize.lemmatize(word) for word in q.split()])

    
    return q

In [12]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [14]:
new_df['q1_len'] = new_df['question1'].str.len() 
new_df['q2_len'] = new_df['question2'].str.len()

In [15]:
new_df['q1_num_words'] = new_df['question1'].apply(lambda row: len(row.split(" ")))
new_df['q2_num_words'] = new_df['question2'].apply(lambda row: len(row.split(" ")))
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7
345040,345040,473326,473327,what is the best rza verse,what are the craziest rap verse,0,26,31,6,6
30329,30329,56030,56031,which programming language should have to lear...,what programming language should i learn for m...,1,66,59,10,9


In [16]:
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return len(w1 & w2)

In [17]:
new_df['word_common'] = new_df.apply(common_words, axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,5
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,4
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,7
345040,345040,473326,473327,what is the best rza verse,what are the craziest rap verse,0,26,31,6,6,3
30329,30329,56030,56031,which programming language should have to lear...,what programming language should i learn for m...,1,66,59,10,9,7


In [18]:
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))

In [19]:
new_df['word_total'] = new_df.apply(total_words, axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,5,16
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,4,14
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,7,25
345040,345040,473326,473327,what is the best rza verse,what are the craziest rap verse,0,26,31,6,6,3,12
30329,30329,56030,56031,which programming language should have to lear...,what programming language should i learn for m...,1,66,59,10,9,7,19


In [20]:
new_df['word_share'] = round(new_df['word_common']/new_df['word_total'],2)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,5,16,0.31
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,4,14,0.29
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,7,25,0.28
345040,345040,473326,473327,what is the best rza verse,what are the craziest rap verse,0,26,31,6,6,3,12,0.25
30329,30329,56030,56031,which programming language should have to lear...,what programming language should i learn for m...,1,66,59,10,9,7,19,0.37


In [21]:
# Advanced Features
from nltk.corpus import stopwords

def fetch_token_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features

In [22]:
token_features = new_df.apply(fetch_token_features, axis=1)

new_df["cwc_min"]       = list(map(lambda x: x[0], token_features))
new_df["cwc_max"]       = list(map(lambda x: x[1], token_features))
new_df["csc_min"]       = list(map(lambda x: x[2], token_features))
new_df["csc_max"]       = list(map(lambda x: x[3], token_features))
new_df["ctc_min"]       = list(map(lambda x: x[4], token_features))
new_df["ctc_max"]       = list(map(lambda x: x[5], token_features))
new_df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
new_df["first_word_eq"] = list(map(lambda x: x[7], token_features))


In [23]:
new_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,word_total,word_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,...,16,0.31,0.749981,0.599988,0.666644,0.499988,0.624992,0.624992,1.0,1.0
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,...,14,0.29,0.999967,0.599988,0.999900,0.199996,0.999975,0.399996,0.0,1.0
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,...,25,0.28,0.999950,0.285710,0.999980,0.454541,0.999986,0.349998,0.0,1.0


In [26]:
import distance
def fetch_length_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features

In [27]:
length_features = new_df.apply(fetch_length_features, axis=1)

new_df['abs_len_diff'] = list(map(lambda x: x[0], length_features))
new_df['mean_len'] = list(map(lambda x: x[1], length_features))
new_df['longest_substr_ratio'] = list(map(lambda x: x[2], length_features))

In [28]:
new_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,...,0.599988,0.666644,0.499988,0.624992,0.624992,1.0,1.0,0.0,8.0,0.425000
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,...,0.599988,0.999900,0.199996,0.999975,0.399996,0.0,1.0,6.0,7.0,0.807692
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,...,0.285710,0.999980,0.454541,0.999986,0.349998,0.0,1.0,13.0,13.5,0.968750


In [29]:
# Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [30]:
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# Creating new feature columns for fuzzy features
new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [31]:
print(new_df.shape)
new_df.isna().sum()

(50000, 28)


id                      0
qid1                    0
qid2                    0
question1               0
question2               0
is_duplicate            0
q1_len                  0
q2_len                  0
q1_num_words            0
q2_num_words            0
word_common             0
word_total              0
word_share              0
cwc_min                 0
cwc_max                 0
csc_min                 0
csc_max                 0
ctc_min                 0
ctc_max                 0
last_word_eq            0
first_word_eq           0
abs_len_diff            0
mean_len                0
longest_substr_ratio    0
fuzz_ratio              0
fuzz_partial_ratio      0
token_sort_ratio        0
token_set_ratio         0
dtype: int64

In [32]:
import gensim
import os

In [34]:
from gensim.utils import simple_preprocess


new_df['tokens_q1'] = new_df['question1'].apply(lambda x: simple_preprocess(x))
new_df['tokens_q2'] = new_df['question2'].apply(lambda x: simple_preprocess(x))

In [35]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio,tokens_q1,tokens_q2
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,...,1.0,0.0,8.0,0.425000,69,64,64,72,"[how, can, accomplish, more, every, single, day]","[how, do, get, motivated, every, single, day]"
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,...,1.0,6.0,7.0,0.807692,59,88,59,100,"[how, doe, gravitylight, work]","[how, doe, gravitylight, from, deciwatt, work,..."
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,...,1.0,13.0,13.5,0.968750,50,100,50,100,"[why, should, not, vote, for, trump, someone, ...","[why, should, not, vote, for, trump]"
345040,345040,473326,473327,what is the best rza verse,what are the craziest rap verse,0,26,31,6,6,...,1.0,0.0,6.0,0.222222,77,69,74,74,"[what, is, the, best, rza, verse]","[what, are, the, craziest, rap, verse]"
30329,30329,56030,56031,which programming language should have to lear...,what programming language should i learn for m...,1,66,59,10,9,...,0.0,1.0,9.5,0.483333,90,88,90,94,"[which, programming, language, should, have, t...","[what, programming, language, should, learn, f..."


In [62]:
model = gensim.models.Word2Vec(window=10,vector_size=500,
                              min_count=2,
                              workers=4)
model.save("word2vec.model")

In [63]:
model = gensim.models.Word2Vec.load("word2vec.model")

In [64]:
texts = new_df["tokens_q1"].to_list() + new_df["tokens_q1"].to_list()

In [65]:
len(texts)

100000

In [66]:
model.build_vocab(texts)

In [67]:
model.train(texts, total_examples=model.corpus_count, epochs = model.epochs)

(3708033, 5153350)

In [92]:
import pickle
pickle.dump(model,open('w2v.pkl','wb'))

In [68]:
def sent_vec(sent):
    vector_size = model.wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in model.wv:
            ctr += 1
            wv_res += model.wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [69]:
new_df['q1_vec'] = new_df['tokens_q1'].apply(sent_vec)
new_df['q2_vec'] = new_df['tokens_q2'].apply(sent_vec)

In [70]:
new_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio,tokens_q1,tokens_q2,q1_vec,q2_vec
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,...,8.0,0.425000,69,64,64,72,"[how, can, accomplish, more, every, single, day]","[how, do, get, motivated, every, single, day]","[-0.1733398144133389, 0.04703097231686115, -0....","[-0.24648437881842256, 0.2664277320727706, 0.1..."
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,...,7.0,0.807692,59,88,59,100,"[how, doe, gravitylight, work]","[how, doe, gravitylight, from, deciwatt, work,...","[-0.08649592027068138, 0.3276750836521387, 0.1...","[-0.16437487097250092, 0.31906578462156987, 0...."
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,...,13.5,0.968750,50,100,50,100,"[why, should, not, vote, for, trump, someone, ...","[why, should, not, vote, for, trump]","[0.06095271693591617, 0.5000824222439214, 0.12...","[0.03290390649012157, 0.5379549052034106, 0.15..."


In [71]:
df1 = pd.DataFrame(new_df['q1_vec'].tolist(), columns=[f'col_{i+1}' for i in range(500)],index = new_df.index)
df2 = pd.DataFrame(new_df['q2_vec'].tolist(), columns=[f'col_{i+501}' for i in range(500)],index = new_df.index)

In [72]:
new_df.index

Int64Index([162260, 300139, 169568, 345040,  30329, 352146, 125219,  38563,
            107116, 111402,
            ...
            351441, 141663, 237712, 278649,  38503, 204760, 162810, 126921,
            106157, 262197],
           dtype='int64', length=50000)

In [73]:
final_df = pd.concat([new_df, df1, df2], axis =1)

In [74]:
print(final_df.shape)
final_df.head()

(50000, 1032)


,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,col_991,col_992,col_993,col_994,col_995,col_996,col_997,col_998,col_999,col_1000
162260,162260,252637,78559,how can i accomplish more every single day,how do i get motivated every single day,0,42,39,8,8,...,-0.032438,-0.210139,-0.041577,0.175258,-0.060471,0.276103,0.104550,-0.340571,0.082752,-0.125160
300139,300139,422897,422898,how doe gravitylight work,how doe gravitylight from deciwatt work what i...,1,25,60,4,10,...,-0.051296,-0.288944,0.045061,0.092399,-0.232579,0.107146,-0.040327,-0.344912,0.024275,-0.161863
169568,169568,187323,74117,why should i not vote for trump someone asked ...,why should i not vote for trump,1,93,31,20,7,...,0.469147,-0.206977,-0.175354,-0.207862,-0.199804,-0.120512,-0.096699,-1.125570,-0.093060,-0.501677
345040,345040,473326,473327,what is the best rza verse,what are the craziest rap verse,0,26,31,6,6,...,-0.148475,0.282985,0.273866,-0.038079,0.369774,-0.029135,0.098392,-0.057253,-0.123080,-0.095333
30329,30329,56030,56031,which programming language should have to lear...,what programming language should i learn for m...,1,66,59,10,9,...,0.068333,0.094804,0.275918,0.144474,0.473276,-0.056963,0.119073,-0.484836,-0.136929,-0.287323


In [75]:
model_df = final_df.drop(columns=['question1', 'question2','id','qid1','qid2', 'tokens_q1', 'tokens_q2', 'q1_vec', 'q2_vec'])

In [94]:
model_df.shape

(50000, 1023)

In [77]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(model_df.iloc[:,1:].values,model_df.iloc[:,0].values,test_size=0.2,random_state=1)


In [140]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
mode_l = RandomForestClassifier()
mode_l.fit(X_train,y_train)
y_pred = mode_l.predict(X_test)
accuracy_score(y_test,y_pred)

0.7904

In [85]:
rf = RandomForestClassifier(n_estimators = 200, max_depth = 9)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7673

In [123]:
from xgboost import XGBClassifier
xgb = XGBClassifier(eval_metric = 'logloss')
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.7926

In [117]:
xgb = XGBClassifier(learning_rate=0.2, eval_metric='logloss', n_estimators=200)
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.8025

In [153]:
mode_l1 = XGBClassifier(learning_rate=0.1, eval_metric='logloss', n_estimators=300, max_depth= 9)
mode_l1.fit(X_train,y_train)
y_pred1 = mode_l1.predict(X_test)
accuracy_score(y_test,y_pred1)

0.8105

In [86]:
def test_common_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return len(w1 & w2)

In [87]:
def test_total_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return (len(w1) + len(w2))


In [88]:
def test_fetch_token_features(q1,q2):
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features


In [152]:
STOP_WORDS = stopwords.words("english")
pickle.dump(STOP_WORDS,open('stop_words.pkl','wb'))

In [89]:
def test_fetch_length_features(q1,q2):
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features


In [90]:
def test_fetch_fuzzy_features(q1,q2):
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features


In [93]:
def sent_vec(sent):
    vector_size = model.wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in model.wv:
            ctr += 1
            wv_res += model.wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [108]:
def query_point_creator(q1,q2):
    
    input_query = []
    
    # preprocess
    q1 = preprocess(q1)
    q2 = preprocess(q2)
    
    # fetch basic features
    input_query.append(len(q1))
    input_query.append(len(q2))
    
    input_query.append(len(q1.split(" ")))
    input_query.append(len(q2.split(" ")))
    
    input_query.append(test_common_words(q1,q2))
    input_query.append(test_total_words(q1,q2))
    input_query.append(round(test_common_words(q1,q2)/test_total_words(q1,q2),2))
    
    # fetch token features
    token_features = test_fetch_token_features(q1,q2)
    input_query.extend(token_features)
    
    # fetch length based features
    length_features = test_fetch_length_features(q1,q2)
    input_query.extend(length_features)
    
    # fetch fuzzy features
    fuzzy_features = test_fetch_fuzzy_features(q1,q2)
    input_query.extend(fuzzy_features)
    
    q1_token = simple_preprocess(q1)
    q2_token = simple_preprocess(q2)
    
    
    q1_bow = sent_vec(q1_token).tolist()
    q2_bow = sent_vec(q2_token).tolist()
    r = input_query + q1_bow + q2_bow
    
    
    
    
    return np.array(r).reshape(1,1022)

In [136]:
q1 = 'What is the capital of India?'
q2 = 'What is the current capital of Pakistan?'
q3 = 'Which city serves as the capital of India?'
q4 = 'What is the business capital of India?'
q5 = 'who is the P.M of India?'
q6 = 'who is the current prime minister of india?'


In [141]:
mode_l.predict(query_point_creator(q3,q4))

array([1])

In [142]:
query_point_creator(q1,q4)

array([[ 2.80000000e+01,  3.70000000e+01,  6.00000000e+00, ...,
        -7.08175290e-02,  1.62215815e-02, -5.87041639e-02]])

In [155]:
pickle.dump(mode_l,open('model.pkl','wb'))

In [156]:
pickle.dump(mode_l1,open('model1.pkl','wb'))

In [151]:
import sklearn
print(sklearn.__version__)

1.2.1
